In [62]:
!pip freeze

^C
Traceback (most recent call last):
  File "/project/envs/qwen-train-kernel/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/project/envs/qwen-train-kernel/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/project/envs/qwen-train-kernel/lib/python3.10/site-packages/pip/_internal/cli/autocompletion.py", line 12, in <module>
    from pip._internal.metadata import get_default_environment
  File "/project/envs/qwen-train-kernel/lib/python3.10/site-packages/pip/_internal/metadata/__init__.py", line 3, in <module>
    from .base import BaseDistribution, BaseEnvironment, FilesystemWheel, MemoryWheel, Wheel
  File "/project/envs/qwen-train-kernel/lib/python3.10/site-packages/pip/_internal/metadata/base.py", line 21, in <module>
    from pip._vendor.packaging.requirements import Requirement
  File "/project/envs/qwen-train-kernel/lib/python3.10/site-packages/pip/

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

qwen_models = [
    "unsloth/Qwen2.5-Coder-32B-Instruct",      # Qwen 2.5 Coder 2x faster
    "unsloth/Qwen2.5-Coder-7B",
    "unsloth/Qwen2.5-14B-Instruct",            # 14B fits in a 16GB card
    "unsloth/Qwen2.5-7B",
    "unsloth/Qwen2.5-72B-Instruct",            # 72B fits in a 48GB card
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/project/envs/qwen-train-kernel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.49.0.
   \\   /|    NVIDIA H100 NVL. Num GPUs = 3. Max memory: 93.016 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
from datasets import load_dataset, Dataset
import pandas as pd

In [4]:
def mapper(x):
    clean_chat = []
    x = eval(x)
    for ll in x:
        if len(ll.keys()) != 2:
            clean_chat.append({
                'role': ll['role'],
                'content': ll['content'],
            })
        else:
            clean_chat.append(ll)
    return clean_chat

In [5]:
df = pd.read_csv("/home/users/nhorst/clem-project_playpen/data/training_data/DFINAL_VTrain_qwen-coder.csv")
df["conversations"] = df["chat"].map(lambda x: mapper(x))
dataset: Dataset = Dataset.from_pandas(df=df)

df_test = pd.read_csv("/home/users/nhorst/clem-project_playpen/data/training_data/DFINAL_VTest_qwen-coder.csv")
df_test["conversations"] = df_test["chat"].map(lambda x: mapper(x))
dataset_test: Dataset = Dataset.from_pandas(df=df_test)


In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [7]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

dataset_test = standardize_sharegpt(dataset_test)
dataset_test = dataset_test.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 291/291 [00:00<00:00, 7057.61 examples/s]


In [8]:
dataset[5]["conversations"]

[{'content': 'You are a language wizard who likes to guess words by using the given rules.\n\nWelcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.\n\nTo help you make an informed guess, you will receive a clue for the word, such as\nclue: "snowy white"\n\nHere is an example guess based on the clue:\nguess: apple\nexplanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.\n\nI will then indicate whether I agree or disagree with your guess and provide rationale, but agreeing with a guess does not confirm its correctness. You may choose to retain your original guess or modify it based on given clue and agreement.\n\nAfter each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are

In [9]:
dataset[5]["text"]

'<s>[INST] You are a language wizard who likes to guess words by using the given rules.\n\nWelcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.\n\nTo help you make an informed guess, you will receive a clue for the word, such as\nclue: "snowy white"\n\nHere is an example guess based on the clue:\nguess: apple\nexplanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.\n\nI will then indicate whether I agree or disagree with your guess and provide rationale, but agreeing with a guess does not confirm its correctness. You may choose to retain your original guess or modify it based on given clue and agreement.\n\nAfter each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are co

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    #eval_dataset= dataset_test,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 1, # Fixed major bug in latest Unsloth
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 1700,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        eval_steps=100,
        optim = "adamw_8bit", # Save more memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 7331,
        output_dir = "../outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Tokenizing to ["text"]: 100%|██████████| 7079/7079 [00:02<00:00, 3537.46 examples/s]


In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "[INST]",
    response_part = "[/INST]",
)

Map (num_proc=64): 100%|██████████| 7079/7079 [00:04<00:00, 1516.04 examples/s]


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,079 | Num Epochs = 3 | Total steps = 1,700
O^O/ \_/ \    Batch size per device = 12 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (12 x 1 x 1) = 12
 "-____-"     Trainable parameters = 167,772,160/3,926,134,784 (4.27% trained)


Step,Training Loss
10,0.664800
20,0.461000
30,0.462800
40,0.424000
50,0.316700
60,0.362100


Unsloth: Will smartly offload gradients to save VRAM!


KeyboardInterrupt: 

In [75]:
print("""<s>[INST] You are a language wizard who likes to guess words by using the given rules.

Welcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.

To help you make an informed guess, you will receive a clue for the word, such as
clue: "snowy white"

Here is an example guess based on the clue:
guess: apple
explanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.

I will then indicate whether I agree or disagree with your guess and provide rationale, but agreeing with a guess does not confirm its correctness. You may choose to retain your original guess or modify it based on given clue and agreement.

After each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are correct but in the wrong position (yellow), and which letters are incorrect (red). This feedback can be useful in determining which letters to include or exclude in your next guess.

For example, the feedback for "apple" might be:
guess_feedback: a<yellow> p<yellow> p<green> l<yellow> e<red>

Here are some rules to keep in mind:
1. Please do not generate any guess feedback or clue on your own
2. Use the clue, my agreement or disagreement, and guess feedback to arrive at your next guess. This should be reflected in your explanation as well.

Let's begin with your first guess.

clue:bronze place
 [/INST]guess: third
explanation: The bronze medal is awarded to the third-place finisher in competitions, so "third" aligns with the clue "bronze place" and is a five-letter word.</s>[INST] clue:bronze place
guess_agreement:yes
agreement_explanation:your guess "third" fits the clue "bronze place" because the bronze medal is awarded to the third-place finisher, and "third" is a five-letter word.
 [/INST]guess: medal
explanation: Although "third" fits the clue "bronze place" and you agreed with it, another five-letter word associated with bronze awards is "medal." Since bronze medals are given to third-place finishers, "medal" also aligns with the clue. I'll guess "medal" this time.</s>[INST] guess_feedback: m<red> e<red> d<yellow> a<red> l<red> [/INST]guess: third
explanation: Based on the clue "bronze place" and your agreement with my initial guess "third," I believe "third" is still the best guess. From the feedback on "medal," we know that:

- The letter **D** is in the word but not in position 3 (since it was yellow when in position 3).
- Letters **M**, **E**, **A**, and **L** are not in the word.

In "third," the **D** is in position 5, which fits the feedback. Also, all the letters in "third" are valid since they don't include the eliminated letters. Therefore, I'll guess "third" again.</s>
""")

<s>[INST] You are a language wizard who likes to guess words by using the given rules.

Welcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.

To help you make an informed guess, you will receive a clue for the word, such as
clue: "snowy white"

Here is an example guess based on the clue:
guess: apple
explanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.

I will then indicate whether I agree or disagree with your guess and provide rationale, but agreeing with a guess does not confirm its correctness. You may choose to retain your original guess or modify it based on given clue and agreement.

After each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are correct but in t